Ta funkcija za izbran mu in smer  z SA poišče grafe za V= (10, 30)

podatke vzame za iz 2del_subpath_mail_grafi.csv, rezultate pa zapisuje v "2del_subpath_SA.csv"

In [ ]:

import random as pyrandom
from sage.all import *
import pandas as pd
import os
import time

# ============================================================
#  SUBPATH NUMBER (OPTIMIZIRANO)
# ============================================================

def subpath_number(G):
    V = list(G.vertices())
    idx = {v: i for i, v in enumerate(V)}
    n = len(V)

    counts = [[0] * n for _ in range(n)]

    for s in V:
        s_idx = idx[s]

        def dfs(u, visited):
            u_idx = idx[u]
            counts[s_idx][u_idx] += 1

            for nei in G.neighbors(u):
                if nei not in visited:
                    visited.add(nei)
                    dfs(nei, visited)
                    visited.remove(nei)

        visited = set([s])
        dfs(s, visited)

    total = 0
    for i in range(n):
        for j in range(i, n):
            total += counts[i][j]

    return total
###############################

def clean_csv_nan(filename):
    # preberi CEL csv kot stringe
    df = pd.read_csv(filename, dtype=str, encoding="utf-8", keep_default_na=False)

    # zamenjaj vse oblike "nan"
    df = df.replace({
        "nan": "",
        "NaN": "",
        "None": "",
        "NONE": "",
        "Null": "",
        "NULL": "",
        "<NA>": "",
        "pd.NA": ""
    }, regex=False)

    # shrani celoten df nazaj
    df.to_csv(filename, index=False, encoding="utf-8")

# ============================================================
#  SHARANJE OPTIMUMA v CSV (popolnoma popravljeno)
# ============================================================
def save_to_csv(n, mu, graph, score, direction):
    filename = "2del_subpath_SA.csv"

    os.makedirs("slike_min", exist_ok=True)
    os.makedirs("slike_max", exist_ok=True)

    g6 = str(graph.graph6_string())

    # Če CSV ne obstaja, ga ustvari brez NA vrednosti
    if not os.path.exists(filename):
        df = pd.DataFrame(columns=[
            "n", "µ(G)",
            "min p_n(G)", "max p_n(G)",
            "slika_min", "slika_max",
            "graph6_min", "graph6_max"
        ])
        df.to_csv(filename, index=False, encoding="utf-8")

    # Beremo brez NA pretvorbe
    df = pd.read_csv(filename, dtype=str, encoding="utf-8", keep_default_na=False)

    needed = [
        "n", "µ(G)",
        "min p_n(G)", "max p_n(G)",
        "slika_min", "slika_max",
        "graph6_min", "graph6_max"
    ]

    for col in needed:
        if col not in df.columns:
            df[col] = ""

    n_str = str(int(n))
    mu_str = str(int(mu))

    mask = (df["n"] == n_str) & (df["µ(G)"] == mu_str)

    # ===================================================
    # 1) NOVA VRSTICA
    # ===================================================
    if not mask.any():
        new_row = {
            "n": n_str,
            "µ(G)": mu_str,

            "min p_n(G)": str(score) if direction == "min" else "",
            "max p_n(G)": str(score) if direction == "max" else "",

            "slika_min": f"slike_min/graf_min_mu{mu}_n{n}.png" if direction == "min" else "",
            "slika_max": f"slike_max/graf_max_mu{mu}_n{n}.png" if direction == "max" else "",

            "graph6_min": g6 if direction == "min" else "",
            "graph6_max": g6 if direction == "max" else ""
        }

        # Shranimo sliko
        if direction == "min":
            graph.plot().save(new_row["slika_min"])
        else:
            graph.plot().save(new_row["slika_max"])

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        df.to_csv(filename, index=False, encoding="utf-8")
        return

    # ===================================================
    # 2) UPDATE OBSTOJEČE VRSTICE
    # ===================================================
    idx = df.index[mask].tolist()[0]

    if direction == "min":
        old = df.at[idx, "min p_n(G)"]

        if old == "" or score < float(old):
            df.at[idx, "min p_n(G)"] = str(score)
            df.at[idx, "graph6_min"] = g6

            slika = f"slike_min/graf_min_mu{mu}_n{n}.png"
            graph.plot().save(slika)
            df.at[idx, "slika_min"] = slika

    else:
        old = df.at[idx, "max p_n(G)"]

        if old == "" or score > float(old):
            df.at[idx, "max p_n(G)"] = str(score)
            df.at[idx, "graph6_max"] = g6

            slika = f"slike_max/graf_max_mu{mu}_n{n}.png"
            graph.plot().save(slika)
            df.at[idx, "slika_max"] = slika

    df.to_csv(filename, index=False, encoding="utf-8")



from sage.all import Graph

# ============================================================
#  MUTATE GRAPH
# ============================================================

def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H


# ============================================================
#  DODAJ 1 VOZLIŠČE
# ============================================================

def add_one_vertex_and_connect(G_old, direction):
    H = Graph(G_old)

    verts = H.vertices()
    new_v = max(verts) + 1
    H.add_vertex(new_v)

    degrees = {v: H.degree(v) for v in H.vertices() if v != new_v}

    if direction == "min":
        anchor = min(degrees, key=degrees.get)
    else:
        anchor = max(degrees, key=degrees.get)

    H.add_edge(anchor, new_v)
    return H


# ============================================================
#  DODAJ 1 POVEZAVO (najbližji non-edge)
# ============================================================

def add_one_edge(G_old):
    H = Graph(G_old)

    V = H.vertices()
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if not H.has_edge(u, v):
                H.add_edge(u, v)
                return H

    raise ValueError("Graf je že poln.")


# ============================================================
#  LOAD G(n,µ) IZ TVOJEGA CSV — POPRAVLJENO
# ============================================================

def load_G(n_val, mu_val, direction):
    df = pd.read_csv("2del_subpath_SA.csv", encoding="utf-8")

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    # Konverzija Sage Integer → Python int
    n_val = int(n_val)
    mu_val = int(mu_val)

    subset = df[df["n"] == n_val]
    if subset.empty:
        raise ValueError(f"Ni vrstic za n={n_val}")

    row = subset[subset["µ(G)"] == mu_val]
    if row.empty:
        raise ValueError(f"Ni vrstice za n={n_val}, µ={mu_val}")

    # KLJUČNO — prisili Python int indeks
    row = row.iloc[int(0)]      # ← EDINA PRAVA OBLIKA

    # graph6
    if direction == "min":
        g6 = str(row["graph6_min"]).strip()
    else:
        g6 = str(row["graph6_max"]).strip()

    if g6 == "":
        raise ValueError(f"Prazni graph6 zapis pri n={n_val}, µ={mu_val}")

    return Graph(g6)



# ============================================================
#  SIMULATED ANNEALING
# ============================================================

def simulated_annealing(n, m, direction,
                        T_start=20.0, T_end=0.001,
                        cooling=0.995, max_steps=N,
                        initial_graph=None):

    G = Graph(initial_graph)

    best_G = G.copy()
    best_score = subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):
        new_G = mutate_graph(current_G)
        new_score = subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G = new_G
            current_score = new_score
        else:
            if pyrandom.random() < exp(-delta/T):
                current_G = new_G
                current_score = new_score

        improved = (
            (direction == "min" and current_score < best_score) or
            (direction == "max" and current_score > best_score)
        )

        if improved:
            best_G = current_G.copy()
            best_score = current_score

        T *= cooling
        if T < T_end:
            break

    return best_G, best_score


# ============================================================
#  GONILNA FUNKCIJA
# ============================================================

def compute_all_for_fixed_mu(mu_fix, direction, n_start, n_end, N):

    for n in range(n_start, n_end + 1):

        print(f"Obdelujem: n={n}, µ={mu_fix}")

        m_target = mu_fix + n - 1

        # ============================================================
        # IZBERI ZAČETNI GRAF
        # ============================================================

        if n == n_start:
            # Za prvi n moraš ročno dati začetni graf ali ga naložiti
            try:
                Gstart = load_G(n, mu_fix, direction)
            except:
                raise ValueError(f"Manjka začetni graf za n={n}, µ={mu_fix}.")
        
        else:
            # vzamemo optimum za n-1
            G_prev = load_G(n-1, mu_fix, direction)

            # dodamo 1 vozlišče
            Gstart = add_one_vertex_and_connect(G_prev, direction)

            # prilagodimo število povezav, da ustreza m = µ + n - 1
            while Gstart.size() < m_target:
                Gstart = add_one_edge(Gstart)

            # če je preveč povezav → odstrani kak edge naključno
            while Gstart.size() > m_target:
                edges = list(Gstart.edges())
                e = pyrandom.choice(edges)
                Gstart.delete_edge(e)
                if not Gstart.is_connected():
                    Gstart.add_edge(e)

        # ============================================================
        # SIMULATED ANNEALING
        # ============================================================

        best_graph, best_value = simulated_annealing(
            n=n,
            m=m_target,
            direction=direction,
            initial_graph=Gstart,
            max_steps=N
        )

        print(f"=== Rezultat za n={n}, µ={mu_fix} ===")
        print("Best:", best_value)
        print("graph6:", best_graph.graph6_string())

        save_to_csv(n, mu_fix, best_graph, best_value, direction)



# ============================================================
#  START
# ============================================================

compute_all_for_fixed_mu(
    mu_fix=8,
    direction="max",
    n_start=9,
    n_end=30,
    N=2000
)



UGOTOVITEV: ZA MIN funkcija ohranja število trikotnih ciklov in samo dodaja liste in naredi verigo ostalih vozlišč.

Sedaj bova pogledala če dobiva boljše približke, če povezavo, ki jo dodamo v graf povežemo od novega vozlišče v vozlišče z največjo stopnjo.

Rezulatati se zapisujejo v "2del_subpath_SA_v_sredisce.csv"

In [ ]:

import random as pyrandom
from sage.all import *
import pandas as pd
import os
import time

# ============================================================
#  SUBPATH NUMBER (OPTIMIZIRANO)
# ============================================================

def subpath_number(G):
    V = list(G.vertices())
    idx = {v: i for i, v in enumerate(V)}
    n = len(V)

    counts = [[0] * n for _ in range(n)]

    for s in V:
        s_idx = idx[s]

        def dfs(u, visited):
            u_idx = idx[u]
            counts[s_idx][u_idx] += 1

            for nei in G.neighbors(u):
                if nei not in visited:
                    visited.add(nei)
                    dfs(nei, visited)
                    visited.remove(nei)

        visited = set([s])
        dfs(s, visited)

    total = 0
    for i in range(n):
        for j in range(i, n):
            total += counts[i][j]

    return total

# ============================================================
#  SHARANJE OPTIMUMA v CSV (popolnoma popravljeno)
# ============================================================
def save_to_csv(n, mu, graph, score, direction):
    filename = "2del_subpath_SA_v_sredisce.csv"

    os.makedirs("slike_min", exist_ok=True)
    os.makedirs("slike_max", exist_ok=True)

    g6 = str(graph.graph6_string())

    # Če CSV ne obstaja, ga ustvari brez NA vrednosti
    if not os.path.exists(filename):
        df = pd.DataFrame(columns=[
            "n", "µ(G)",
            "min p_n(G)", "max p_n(G)",
            "slika_min", "slika_max",
            "graph6_min", "graph6_max"
        ])
        df.to_csv(filename, index=False, encoding="utf-8")

    # Beremo brez NA pretvorbe
    df = pd.read_csv(filename, dtype=str, encoding="utf-8", keep_default_na=False)

    needed = [
        "n", "µ(G)",
        "min p_n(G)", "max p_n(G)",
        "slika_min", "slika_max",
        "graph6_min", "graph6_max"
    ]

    for col in needed:
        if col not in df.columns:
            df[col] = ""

    n_str = str(int(n))
    mu_str = str(int(mu))

    mask = (df["n"] == n_str) & (df["µ(G)"] == mu_str)

    # ===================================================
    # 1) NOVA VRSTICA
    # ===================================================
    if not mask.any():
        new_row = {
            "n": n_str,
            "µ(G)": mu_str,

            "min p_n(G)": str(score) if direction == "min" else "",
            "max p_n(G)": str(score) if direction == "max" else "",

            "slika_min": f"slike_min/graf_min_mu{mu}_n{n}.png" if direction == "min" else "",
            "slika_max": f"slike_max/graf_max_mu{mu}_n{n}.png" if direction == "max" else "",

            "graph6_min": g6 if direction == "min" else "",
            "graph6_max": g6 if direction == "max" else ""
        }

        # Shranimo sliko
        if direction == "min":
            graph.plot().save(new_row["slika_min"])
        else:
            graph.plot().save(new_row["slika_max"])

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        df.to_csv(filename, index=False, encoding="utf-8")
        return

    # ===================================================
    # 2) UPDATE OBSTOJEČE VRSTICE
    # ===================================================
    idx = df.index[mask].tolist()[0]

    if direction == "min":
        old = df.at[idx, "min p_n(G)"]

        if old == "" or score < float(old):
            df.at[idx, "min p_n(G)"] = str(score)
            df.at[idx, "graph6_min"] = g6

            slika = f"slike_min/graf_min_mu{mu}_n{n}.png"
            graph.plot().save(slika)
            df.at[idx, "slika_min"] = slika

    else:
        old = df.at[idx, "max p_n(G)"]

        if old == "" or score > float(old):
            df.at[idx, "max p_n(G)"] = str(score)
            df.at[idx, "graph6_max"] = g6

            slika = f"slike_max/graf_max_mu{mu}_n{n}.png"
            graph.plot().save(slika)
            df.at[idx, "slika_max"] = slika

    df.to_csv(filename, index=False, encoding="utf-8")



from sage.all import Graph

# ============================================================
#  MUTATE GRAPH
# ============================================================

def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H


# ============================================================
#  DODAJ 1 VOZLIŠČE
# ============================================================

def add_one_vertex_and_connect(G_old, direction):
    H = Graph(G_old)

    verts = H.vertices()
    new_v = max(verts) + 1
    H.add_vertex(new_v)

    # izračun stopenj
    degrees = {v: H.degree(v) for v in H.vertices() if v != new_v}

    if direction == "min":
        # pri minimizaciji se povežeš z vozliščem z NAJVEČJO stopnjo
        anchor = max(degrees, key=degrees.get)

    else:  # direction == "max"
        # pri maksimizaciji se povežeš z vozliščem z NAJMANJŠO stopnjo
        anchor = min(degrees, key=degrees.get)

    H.add_edge(anchor, new_v)
    return H

# ============================================================
#  DODAJ 1 POVEZAVO (najbližji non-edge)
# ============================================================

def add_one_edge(G_old):
    H = Graph(G_old)

    V = H.vertices()
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if not H.has_edge(u, v):
                H.add_edge(u, v)
                return H

    raise ValueError("Graf je že poln.")


# ============================================================
#  LOAD G(n,µ) IZ TVOJEGA CSV — POPRAVLJENO
# ============================================================

def load_G(n_val, mu_val, direction):
    df = pd.read_csv("2del_subpath_SA_v_sredisce.csv", encoding="utf-8")

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    # Konverzija Sage Integer → Python int
    n_val = int(n_val)
    mu_val = int(mu_val)

    subset = df[df["n"] == n_val]
    if subset.empty:
        raise ValueError(f"Ni vrstic za n={n_val}")

    row = subset[subset["µ(G)"] == mu_val]
    if row.empty:
        raise ValueError(f"Ni vrstice za n={n_val}, µ={mu_val}")

    # KLJUČNO — prisili Python int indeks
    row = row.iloc[int(0)]      # ← EDINA PRAVA OBLIKA

    # graph6
    if direction == "min":
        g6 = str(row["graph6_min"]).strip()
    else:
        g6 = str(row["graph6_max"]).strip()

    if g6 == "":
        raise ValueError(f"Prazni graph6 zapis pri n={n_val}, µ={mu_val}")

    return Graph(g6)



# ============================================================
#  SIMULATED ANNEALING
# ============================================================

def simulated_annealing(n, m, direction,
                        T_start=25.0, T_end=0.001,
                        cooling=0.995, max_steps=N,
                        initial_graph=None):

    G = Graph(initial_graph)

    best_G = G.copy()
    best_score = subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):
        new_G = mutate_graph(current_G)
        new_score = subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G = new_G
            current_score = new_score
        else:
            if pyrandom.random() < exp(-delta/T):
                current_G = new_G
                current_score = new_score

        improved = (
            (direction == "min" and current_score < best_score) or
            (direction == "max" and current_score > best_score)
        )

        if improved:
            best_G = current_G.copy()
            best_score = current_score

        T *= cooling
        if T < T_end:
            break

    return best_G, best_score


# ============================================================
#  GONILNA FUNKCIJA
# ============================================================

def compute_all_for_fixed_mu(mu_fix, direction, n_start, n_end, N):

    for n in range(n_start, n_end + 1):

        print(f"Obdelujem: n={n}, µ={mu_fix}")

        m_target = mu_fix + n - 1

        # ============================================================
        # IZBERI ZAČETNI GRAF
        # ============================================================

        if n == n_start:
            # Za prvi n moraš ročno dati začetni graf ali ga naložiti
            try:
                Gstart = load_G(n, mu_fix, direction)
            except:
                raise ValueError(f"Manjka začetni graf za n={n}, µ={mu_fix}.")
        
        else:
            # vzamemo optimum za n-1
            G_prev = load_G(n-1, mu_fix, direction)

            # dodamo 1 vozlišče
            Gstart = add_one_vertex_and_connect(G_prev, direction)

            # prilagodimo število povezav, da ustreza m = µ + n - 1
            while Gstart.size() < m_target:
                Gstart = add_one_edge(Gstart)

            # če je preveč povezav → odstrani kak edge naključno
            while Gstart.size() > m_target:
                edges = list(Gstart.edges())
                e = pyrandom.choice(edges)
                Gstart.delete_edge(e)
                if not Gstart.is_connected():
                    Gstart.add_edge(e)

        # ============================================================
        # SIMULATED ANNEALING
        # ============================================================

        best_graph, best_value = simulated_annealing(
            n=n,
            m=m_target,
            direction=direction,
            initial_graph=Gstart,
            max_steps=N
        )

        print(f"=== Rezultat za n={n}, µ={mu_fix} ===")
        print("Best:", best_value)
        print("graph6:", best_graph.graph6_string())

        save_to_csv(n, mu_fix, best_graph, best_value, direction)



# ============================================================
#  START
# ============================================================

compute_all_for_fixed_mu(
    mu_fix=4,
    direction="max",
    n_start=10,
    n_end=30,
    N=2000
)



UGOTOVITEV: za min vrneta oba načina iste vrednosti za subpath number, kar je logično, saj je išto število podpoti , če so vsa ozlišča iz enega "sredinskega" ali pa če se spenjajo v verigo. število vrikotnih ciklov ostanke isto v obeh primerih ( torej ko je mu = 3 bo imel graf z minimalnim subpathom imelo 3 trikotne cikle, ostala vozlišča bodo pa v verigi)

Za max: tudi v tem primeru so funkcije primerljive, torej v določenem primeru vrne ena manjšo vrednost v drugih pa druga.  več vrne tista, ki najde graf, kjer so tiste "nmove" povezave dodane v cikel in ne da so kot listi dodani na neko vozlišče.  to je samo napaka simulated analinga in temu nemoreva nič pomagati. 

skratka, ohranila bova obe funkciji, tako 1., ki doda vozlišče in povezavo ranbdomly noter, kozt 2., ki doda to vozlišče, na tisto z največjo stopnjo. rezulatati so zapisani v za 1. v Za_male_mu_SA.csv,  za drugo pa v fiksen_mu_v_središče.csv.  v mapo pa so shranjene slike iz 2. funkcije do mu= 4,  za mu = 5,6,7,8 pa so shranjene slike narejene iz prve funkcije.